**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [2]:
!pip install scikit-video

     |████████████████████████████████| 2.3MB 2.8MB/s 


In [3]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten
import keras.backend as K

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

In a training setting, the act functions selects either a random action with probability $\epsilon$ and the best action given the current state according to the model with probability $1 - \epsilon$.

Epsilon is critical to explore all the environment and learn what the best actions are. Indeed if $\epsilon = 0$ the model can be confined to a single state and never attempt to take actions that would lead it to another potentially more rewarding state.

The chosen value of epsilon (between 0 and 1) is a trade-off between exploration (trying to test all possible action-states to learn to infer their rewards) and exploitation (maximizing the reward using the best known action at each time step). Typically, we use a decreasing $\epsilon$ so that we explore a lot at early stage of the training, and then exploit once the model has a good knowledge of its environment.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=100 # set small when debugging
epochs_test=25 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The array board indicates stores the position of each cheese and each poison. We first add cheese on the grid, and then the poison. If a poison ends up on a position where there is cheese, we remove the poison.

The position array is used to store the position of the border of the grid. We pad the grid with 2 rows and columns of 'walls' where the rat cannot go so that we do not have any edge case for the vision of the rat which can see 2 squares around him.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0,4)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(0,epochs):

      # At each epoch, we restart to a fresh game and get the initial state
      state = env.reset()
      # This assumes that the games will end
      game_over = False
  
      win = 0
      lose = 0
  
      while not game_over:
          # The agent performs an action
          action = agent.act(state)
  
          # Apply an action to the environment, get the next state, the reward
          # and if the games end
          prev_state = state
          state, reward, game_over = env.act(action)
  
          # Update the counters
          if reward > 0:
              win = win + reward
          if reward < 0:
              lose = lose -reward
      
      # Save as a mp4
      if e % 10 == 0:
          env.draw(e)


      # Save as a mp4
      env.draw(prefix+str(e))

      # Update stats
      score = score + win-lose

      print("Win/lose count {}/{}. Average score ({})"
            .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [10]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 9.5/18.0. Average score (-8.5)
Win/lose count 13.0/17.0. Average score (-6.25)
Win/lose count 9.0/14.0. Average score (-5.833333333333333)
Win/lose count 8.5/13.0. Average score (-5.5)
Win/lose count 5.5/12.0. Average score (-5.7)
Win/lose count 8.5/14.0. Average score (-5.666666666666667)
Win/lose count 9.0/12.0. Average score (-5.285714285714286)
Win/lose count 8.5/8.0. Average score (-4.5625)
Win/lose count 9.0/16.0. Average score (-4.833333333333333)
Win/lose count 11.0/10.0. Average score (-4.25)
Win/lose count 6.0/14.0. Average score (-4.590909090909091)
Win/lose count 13.5/14.0. Average score (-4.25)
Win/lose count 7.0/13.0. Average score (-4.384615384615385)
Win/lose count 7.5/5.0. Average score (-3.892857142857143)
Win/lose count 10.5/15.0. Average score (-3.933333333333333)
Win/lose count 13.0/10.0. Average score (-3.5)
Win/lose count 9.0/12.0. Average score (-3.4705882352941178)
Win/lose count 10.0/16.0. Average score (-3.611111111111111)
Win/lose count 11.0/1

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory)>=self.max_memory:
          self.memory.pop(0)
        self.memory.append(m)

    def random_access(self):
        return self.memory[np.random.randint(0,len(self.memory))]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(1,epoch+1):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))
      

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
      s=s.reshape((1,5,5,self.n_state))
      return np.argmax(self.model.predict([s]))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        next_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        memories=[]
        
        for i in range(self.batch_size):
          memories.append(self.memory.random_access())
          input_states[i]= memories[-1][0]
          next_states[i]=memories[-1][1]

        target_q=self.model.predict(input_states)
        prediction_q=self.model.predict(next_states)

        for i in range(self.batch_size):
          game_over_= memories[i][4]
            
          if game_over_:
              target_q[i,memories[i][2]]=memories[i][3]
          else:
              target_q[i,memories[i][2]]=memories[i][3]+self.discount*np.max(prediction_q[i])
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("adam", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state)))
        model.add(Dense(100,activation="sigmoid"))
        model.add(Dense(50,activation="sigmoid"))
        model.add(Dense(4))
        
        model.compile(adam(lr=lr, decay=1e-6), "mse")
        self.model = model
        

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.0001, epsilon = 0.1, memory_size=2000, batch_size = 64)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train90.mp4'))













Epoch 001/100 | Loss 0.0066 | Win/lose count 3.0/1.0 (2.0)
Epoch 002/100 | Loss 0.0085 | Win/lose count 3.5/6.0 (-2.5)
Epoch 003/100 | Loss 0.0079 | Win/lose count 5.0/2.0 (3.0)
Epoch 004/100 | Loss 0.0053 | Win/lose count 1.0/3.0 (-2.0)
Epoch 005/100 | Loss 0.0059 | Win/lose count 0.5/3.0 (-2.5)
Epoch 006/100 | Loss 0.0063 | Win/lose count 8.0/4.0 (4.0)
Epoch 007/100 | Loss 0.0093 | Win/lose count 2.5/1.0 (1.5)
Epoch 008/100 | Loss 0.0003 | Win/lose count 5.0/0 (5.0)
Epoch 009/100 | Loss 0.0079 | Win/lose count 2.5/6.0 (-3.5)
Epoch 010/100 | Loss 0.0257 | Win/lose count 2.5/2.0 (0.5)
Epoch 011/100 | Loss 0.0214 | Win/lose count 4.0/2.0 (2.0)
Epoch 012/100 | Loss 0.0097 | Win/lose count 3.5/3.0 (0.5)
Epoch 013/100 | Loss 0.0157 | Win/lose count 3.5/3.0 (0.5)
Epoch 014/100 | Loss 0.0064 | Win/lose count 4.0/3.0 (1.0)
Epoch 015/100 | Loss 0.0104 | Win/lose count 7.0/4.0 (3.0)
Epoch 016/100 | Loss 0.0044 | Win/lose count 3.0/4.0 (-1.0)
Epoch 017/100 | Loss 0.0040 | Win/lose co

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)

        model = Sequential()
        model.add(Conv2D(16,2,padding="same",activation="relu"))
        model.add(MaxPooling2D((2,2)))
        model.add(Conv2D(32,2,padding="same",activation="relu"))
        model.add(MaxPooling2D((2,2)))
        model.add(Flatten())
        model.add(Dense(100,activation="sigmoid"))
        model.add(Dense(50,activation="sigmoid"))
        model.add(Dense(4))
        
        
        model.compile(adam(lr=lr, decay=1e-4), "mse")
        self.model = model

In [47]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.0001, epsilon = 0.1, memory_size=2000, batch_size = 128)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train90.mp4'))

Epoch 001/100 | Loss 0.0072 | Win/lose count 6.0/5.0 (1.0)
Epoch 002/100 | Loss 0.0045 | Win/lose count 6.0/3.0 (3.0)
Epoch 003/100 | Loss 0.0129 | Win/lose count 2.5/10.0 (-7.5)
Epoch 004/100 | Loss 0.0132 | Win/lose count 6.5/7.0 (-0.5)
Epoch 005/100 | Loss 0.0095 | Win/lose count 10.0/18.0 (-8.0)
Epoch 006/100 | Loss 0.0136 | Win/lose count 4.5/11.0 (-6.5)
Epoch 007/100 | Loss 0.0183 | Win/lose count 5.0/6.0 (-1.0)
Epoch 008/100 | Loss 0.0088 | Win/lose count 3.0/2.0 (1.0)
Epoch 009/100 | Loss 0.0137 | Win/lose count 3.5/3.0 (0.5)
Epoch 010/100 | Loss 0.0053 | Win/lose count 4.5/2.0 (2.5)
Epoch 011/100 | Loss 0.0085 | Win/lose count 3.0/3.0 (0.0)
Epoch 012/100 | Loss 0.0071 | Win/lose count 8.5/3.0 (5.5)
Epoch 013/100 | Loss 0.0155 | Win/lose count 5.5/1.0 (4.5)
Epoch 014/100 | Loss 0.0076 | Win/lose count 2.0/1.0 (1.0)
Epoch 015/100 | Loss 0.0153 | Win/lose count 5.0/0 (5.0)
Epoch 016/100 | Loss 0.0059 | Win/lose count 4.5/1.0 (3.5)
Epoch 017/100 | Loss 0.0102 | Win/lose count 3.5/

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [48]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.0001, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.0001, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 12.0/1.0. Average score (11.0)
Win/lose count 9.0/2.0. Average score (9.0)
Win/lose count 12.5/2.0. Average score (9.5)
Win/lose count 9.5/1.0. Average score (9.25)
Win/lose count 15.0/0. Average score (10.4)
Win/lose count 16.5/2.0. Average score (11.083333333333334)
Win/lose count 8.0/1.0. Average score (10.5)
Win/lose count 14.5/3.0. Average score (10.625)
Win/lose count 8.0/2.0. Average score (10.11111111111111)
Win/lose count 8.5/1.0. Average score (9.85)
Win/lose count 5.0/5.0. Average score (8.954545454545455)
Win/lose count 4.5/2.0. Average score (8.416666666666666)
Win/lose count 8.0/1.0. Average score (8.307692307692308)
Win/lose count 6.5/4.0. Average score (7.892857142857143)
Win/lose count 26.0/6.0. Average score (8.7)
Win/lose count 7.5/1.0. Average score (8.5625)
Win/lose count 8.5/2.0. Average score (8.441176470588236)
Win/lose count 12.5/1.0. Average score (8.61111111111111)
Win/lose count 11.5/2.0. Average score (8.657894736842104)
Win/l

In [49]:
HTML(display_videos('cnn_test24.mp4'))

In [50]:
HTML(display_videos('fc_test9.mp4'))

We can see that the agent doesn't explore its environment when he doesn't see any cheese to eat. This is a big issue when the temperature is low (i.e. there is little cheese on the grid) and the rat gets stuck in a corner very fast and doesn't pick up all the cheese from the map even if they are easily reachable.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,prefix=''):

    # Number of won games
    score = 0
    loss = 0

    for e in range(1,epoch+1):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over, real_reward = env.act(action, train=True)

            # Update the counters
            if real_reward > 0:
                win = win + real_reward
            if real_reward < 0:
                lose = lose -real_reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            agent.set_epsilon(agent.epsilon-agent.epsilon/20)
            print("Current epsilon value :",agent.epsilon)
            env.draw(prefix+str(e))
        
        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
       super().__init__(grid_size,max_time,temperature)
       self.malus_position=np.zeros((grid_size,grid_size))
    
    def reset(self):
      super().reset()
      self.malus_position=np.zeros((self.grid_size,self.grid_size))

      state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
      

      return state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
    
    def act(self, action, train=False):
        state, reward, game_over = super().act(action)
        real_reward=reward
        if train:
            reward -= self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] += 0.2
        self.malus_position[self.x, self.y]=np.minimum(self.malus_position[self.x, self.y],0.4)

        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state=state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        if train:
          return state,reward,game_over,real_reward
        return state,real_reward,game_over



In [52]:
# Training
epochs_train=200

env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.0001, epsilon = 0.1, memory_size=2000, batch_size = 128,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 001/200 | Loss 0.0360 | Win/lose count 8.0/9.0 (-1.0)
Epoch 002/200 | Loss 0.0410 | Win/lose count 10.0/14.0 (-4.0)
Epoch 003/200 | Loss 0.0058 | Win/lose count 5.0/6.0 (-1.0)
Epoch 004/200 | Loss 0.0169 | Win/lose count 2.0/2.0 (0.0)
Epoch 005/200 | Loss 0.0183 | Win/lose count 3.0/7.0 (-4.0)
Epoch 006/200 | Loss 0.0171 | Win/lose count 6.5/9.0 (-2.5)
Epoch 007/200 | Loss 0.0286 | Win/lose count 4.5/7.0 (-2.5)
Epoch 008/200 | Loss 0.0043 | Win/lose count 11.0/12.0 (-1.0)
Epoch 009/200 | Loss 0.0083 | Win/lose count 12.5/10.0 (2.5)
Current epsilon value : 0.095
Epoch 010/200 | Loss 0.0073 | Win/lose count 11.0/13.0 (-2.0)
Epoch 011/200 | Loss 0.0057 | Win/lose count 11.5/6.0 (5.5)
Epoch 012/200 | Loss 0.0060 | Win/lose count 8.0/7.0 (1.0)
Epoch 013/200 | Loss 0.0053 | Win/lose count 11.0/19.0 (-8.0)
Epoch 014/200 | Loss 0.0063 | Win/lose count 5.0/17.0 (-12.0)
Epoch 015/200 | Loss 0.0312 | Win/lose count 10.0/11.0 (-1.0)
Epoch 016/200 | Loss 0.0221 | Win/lose count 11.5/19.0 (-7.

In [53]:
HTML(display_videos('cnn_train_explore200.mp4'))


In [56]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore24.mp4'))

Win/lose count 27.0/19.0. Average score (8.0)
Win/lose count 18.0/19.0. Average score (3.5)
Win/lose count 19.5/23.0. Average score (1.1666666666666667)
Win/lose count 26.5/14.0. Average score (4.0)
Win/lose count 24.0/18.0. Average score (4.4)
Win/lose count 18.5/19.0. Average score (3.5833333333333335)
Win/lose count 22.0/18.0. Average score (3.642857142857143)
Win/lose count 21.5/22.0. Average score (3.125)
Win/lose count 23.0/18.0. Average score (3.3333333333333335)
Win/lose count 21.5/18.0. Average score (3.35)
Win/lose count 20.5/17.0. Average score (3.3636363636363638)
Win/lose count 15.5/12.0. Average score (3.375)
Win/lose count 23.0/13.0. Average score (3.8846153846153846)
Win/lose count 17.0/11.0. Average score (4.035714285714286)
Win/lose count 19.5/15.0. Average score (4.066666666666666)
Win/lose count 22.5/7.0. Average score (4.78125)
Win/lose count 20.5/10.0. Average score (5.117647058823529)
Win/lose count 29.5/16.0. Average score (5.583333333333333)
Win/lose count 18.0

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***